<a href="https://colab.research.google.com/github/Andre-Hua/Andre-Hua/blob/main/Completed_Fed_CTNN_update_(split_client_first_then_shuffle).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Library and Data Files

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, confusion_matrix, \
    precision_recall_fscore_support, roc_auc_score
from sklearn.metrics import classification_report
from tensorflow.keras import Sequential



from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Activation
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

from keras import backend as K

import math
import random

import matplotlib.pyplot as plt

# TNN library
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, concatenate
from tensorflow.keras import layers, models, optimizers, callbacks
from keras.models import Model
from sklearn.model_selection import train_test_split

from tensorflow.keras.regularizers import l2

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, confusion_matrix, \
    precision_recall_fscore_support, roc_auc_score
from sklearn.utils import shuffle

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, MultiHeadAttention, Flatten, Add
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error, confusion_matrix


from tensorflow.python.ops.parallel_for.pfor import def_function


# plot library
import matplotlib.pyplot as plt

from google.colab import drive

In [2]:
drive.mount('/content/gdrive')
df1 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/FYP/data/df_100u_535d_24ppd_15%_client1.csv")
df2 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/FYP/data/df_100u_535d_24ppd_15%_client2.csv")
df3 = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/FYP/data/df_100u_535d_24ppd_15%_client3.csv")

# df1 = df1.iloc[1:] # drop the first row
# df2 = df2.iloc[1:] # drop the first row
# df3 = df3.iloc[1:] # drop the first row

# print("df1: ", df1)
# print("df2: ", df2)

# df1.pop(df1.columns[0]) # drop the first column (i.e. the wrong index column)
# df2.pop(df2.columns[0]) # drop the first column (i.e. the wrong index column)
# df3.pop(df3.columns[0]) # drop the first column (i.e. the wrong index column)

# print("df1: ", df1)
# print("df2: ", df2)

print(df1)
print(df2)

Mounted at /content/gdrive
       Unnamed: 0        c1        c2        c3        c4        c5        c6  \
0               0  0.208787  0.062782  0.062782  0.106584  0.063512  0.168636   
1               1  0.210726  0.359244  0.066872  0.067650  0.109639  0.171846   
2               2  0.051645  0.160217  0.068664  0.049884  0.062209  0.075707   
3               3  0.161297  0.043674  0.075437  0.043178  0.043674  0.138467   
4               4  0.024164  0.039566  0.022837  0.023102  0.038504  0.055498   
...           ...       ...       ...       ...       ...       ...       ...   
53599       53599  1.333000  1.070000  0.938000  0.900000  0.819000  0.317000   
53600       53600  3.155000  1.892000  2.177000  0.715000  0.405000  0.071000   
53601       53601  1.278000  1.149000  0.879000  0.955000  0.715000  0.065000   
53602       53602  1.434000  1.050000  0.864000  1.038000  0.133000  0.072000   
53603       53603  1.092000  1.420000  1.304000  1.060000  0.980000  0.472000   



## Create train and test dataset

### Split into train and test dataset for each client

In [3]:
def read_data(X, y, train_size):

    # Splitting the dataset into training set and test set
    # X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state = None, shuffle = True, stratify = None)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state = None, shuffle = True, stratify = y)
    # no need to shuffle again as df has been shuffled previously

    return X_train, X_test, y_train, y_test

In [4]:
train_size = 0.6  # percentage of the test size comparing to the whole dataset

X1 = df1.iloc[:, 0:24] # drop the Type and Label column
y1 = df1['Label']

X2 = df2.iloc[:, 0:24] # drop the Type and Label column
y2 = df2['Label']

X3 = df3.iloc[:, 0:24] # drop the Type and Label column
y3 = df3['Label']


X_train1, X_test1, y_train1, y_test1 = read_data(X1, y1, train_size)
X_train2, X_test2, y_train2, y_test2 = read_data(X2, y2, train_size)
X_train3, X_test3, y_train3, y_test3 = read_data(X3, y3, train_size)

### Combine each client's train dataset to get data batch and combine test dataset for global testing

In [5]:
X_train = pd.concat([X_train1, X_train2, X_train3])
print(X_train)

y_train = pd.concat([y_train1, y_train2, y_train3])

X_test = pd.concat([X_test1, X_test2, X_test3])

y_test = pd.concat([y_test1, y_test2, y_test3])

print("X_train: ", X_train)
print("X_test: ", X_test)

# label_list = df['Label']

# data_list = df.iloc[:, 0:24]


# print(data_list)
# print(label_list)

       Unnamed: 0     c1     c2     c3     c4     c5     c6      c7      c8  \
28143       28143  0.739  0.384  0.328  0.377  0.192  0.132   0.171   0.144   
46084       46084  1.554  0.326  0.352  1.008  2.233  1.160   0.569   0.518   
18790       18790  0.276  0.093  0.414  0.256  0.244  2.239   0.732   0.397   
6084         6084  0.856  1.891  0.802  3.656  1.154  1.642   1.612   1.560   
10361       10361  4.230  4.254  4.192  4.798  4.107  5.952  17.405  16.002   
...           ...    ...    ...    ...    ...    ...    ...     ...     ...   
1264         1264  0.000  0.005  0.009  0.000  0.000  0.000   0.000   0.000   
4782          762  2.137  2.338  1.706  1.657  2.884  3.068   0.683   0.481   
8670         8670  0.867  0.492  1.476  0.869  0.530  0.364   0.855   1.588   
25505       25505  0.189  0.198  0.227  0.253  0.271  0.243   1.506   0.624   
2639         2639  0.544  0.481  0.417  0.357  0.008  0.018   0.064   0.059   

           c9  ...    c14    c15    c16    c17    c

In [6]:
print("y_train: ", y_train)
print("y_test: ", y_test)

y_train:  28143    0
46084    0
18790    0
6084     0
10361    0
        ..
1264     1
4782     1
8670     0
25505    0
2639     1
Name: Label, Length: 96486, dtype: int64
y_test:  48302    0
13447    0
50670    0
42873    0
49797    0
        ..
30336    0
20747    0
42282    0
19936    0
41882    0
Name: Label, Length: 64326, dtype: int64


In [7]:
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# Split the data into input and target variables
X_train = X_train.values
X_test= X_test.values

y_train = y_train.values
y_test = y_test.values

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print("X_train: ", X_train)
print("X_test: ", X_test)
print("y_train: ", y_train)

X_train:  [[[2.8143e+04]
  [7.3900e-01]
  [3.8400e-01]
  ...
  [4.2900e-01]
  [6.0400e-01]
  [6.3500e-01]]

 [[4.6084e+04]
  [1.5540e+00]
  [3.2600e-01]
  ...
  [1.2300e-01]
  [1.1900e-01]
  [4.7400e-01]]

 [[1.8790e+04]
  [2.7600e-01]
  [9.3000e-02]
  ...
  [4.3000e-01]
  [3.2700e-01]
  [1.0400e-01]]

 ...

 [[8.6700e+03]
  [8.6700e-01]
  [4.9200e-01]
  ...
  [1.5400e-01]
  [1.0300e-01]
  [3.8300e-01]]

 [[2.5505e+04]
  [1.8900e-01]
  [1.9800e-01]
  ...
  [6.0300e-01]
  [1.5210e+00]
  [5.2600e-01]]

 [[2.6390e+03]
  [5.4400e-01]
  [4.8100e-01]
  ...
  [0.0000e+00]
  [0.0000e+00]
  [0.0000e+00]]]
X_test:  [[[4.8302e+04]
  [3.5800e-01]
  [4.1300e-01]
  ...
  [8.5000e-02]
  [9.9000e-02]
  [1.0900e-01]]

 [[1.3447e+04]
  [6.2000e-01]
  [6.7000e-01]
  ...
  [4.6000e-01]
  [5.2900e-01]
  [5.8200e-01]]

 [[5.0670e+04]
  [1.3100e-01]
  [7.9000e-02]
  ...
  [8.7700e-01]
  [6.0600e-01]
  [7.6000e-02]]

 ...

 [[4.2282e+04]
  [1.0500e-01]
  [1.6810e+00]
  ...
  [2.0000e-01]
  [3.2000e-01]
  [9.9

## Fed Avg Algorithm


In [8]:
# fed avg algorithm for model aggregation
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights
       input: weight
       output: final scaled weight'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of all clients' scaled weights. The is equivalent to scaled avg of the weights
       input: scaled weight list (i.e. the output of function "scale_model_weights" -> weight_final list)
       output: the average weights over all clients'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

## Creating Client that have its own training and test dataset

In [9]:
def create_clients(data_list, label_list, X_test, y_test, num_clients, initial='clients'):
    """
    Returns a dictionary with keys as clients' names and values as data shards - tuples of
    training data, training labels, test data, and test labels.

    Args:
        data_list: A list of numpy arrays of training data (consumption data).
        label_list: A list of binarized labels for the training data.
        X_test: Test features as a numpy array.
        y_test: Test labels as a numpy array.
        num_clients: Number of federated members (clients).
        initial: The clients' name prefix, e.g., 'client_1'.
    """
    # Create a list of client names
    client_names = [f'{initial}_{i+1}' for i in range(num_clients)]

    # Ensure data_list and label_list are properly shaped arrays
    assert len(data_list) == len(label_list), "Mismatch in lengths of data_list and label_list"

    # Calculate sizes for sharding
    total_samples = len(data_list)
    size_per_client = total_samples // num_clients

    # Shard the training data and labels
    clients = {}
    for i, client_name in enumerate(client_names):
        start_index = i * size_per_client
        end_index = start_index + size_per_client
        client_data = data_list[start_index:end_index]
        client_labels = label_list[start_index:end_index]

        # Shard the test data and labels similarly if needed or distribute uniformly
        test_start = i * (len(X_test) // num_clients)
        test_end = test_start + (len(X_test) // num_clients)

        clients[client_name] = {
            'train_data': (np.array(client_data), np.array(client_labels)),
            'test_data': (X_test[test_start:test_end], y_test[test_start:test_end])
        }

    return clients


### Create number of local centers (i.e. # of clients)


In [10]:
# Create clients with both training and test data
num_clients = 3
clients = create_clients(X_train, y_train, X_test, y_test, num_clients=num_clients, initial='client')

In [11]:
# Printing out the content in each client to check and debug
for client_name, data in clients.items():
    # Print client name
    print(f"Client: {client_name}")

    # Print a small portion of the training data and labels
    print("Training data samples:")
    print(data['train_data'][0][:2])  # Print first 2 samples from training data
    print("Training labels samples:")
    print(data['train_data'][1][:2])  # Print first 2 labels from training data

    # Print a small portion of the test data and labels
    print("Test data samples:")
    print(data['test_data'][0][:2])  # Print first 2 samples from test data
    print("Test labels samples:")
    print(data['test_data'][1][:2])  # Print first 2 labels from test data
    print("\n")  # Print newline for better readability between clients


Client: client_1
Training data samples:
[[[2.8143e+04]
  [7.3900e-01]
  [3.8400e-01]
  [3.2800e-01]
  [3.7700e-01]
  [1.9200e-01]
  [1.3200e-01]
  [1.7100e-01]
  [1.4400e-01]
  [1.3800e-01]
  [1.5300e-01]
  [1.6600e-01]
  [1.2800e-01]
  [3.1300e-01]
  [6.2900e-01]
  [1.2500e-01]
  [1.7400e-01]
  [1.5700e-01]
  [1.4400e-01]
  [1.4900e-01]
  [1.6000e-01]
  [4.2900e-01]
  [6.0400e-01]
  [6.3500e-01]]

 [[4.6084e+04]
  [1.5540e+00]
  [3.2600e-01]
  [3.5200e-01]
  [1.0080e+00]
  [2.2330e+00]
  [1.1600e+00]
  [5.6900e-01]
  [5.1800e-01]
  [9.1300e-01]
  [8.7200e-01]
  [9.9700e-01]
  [1.8520e+00]
  [5.8300e-01]
  [3.7100e-01]
  [2.8000e-01]
  [2.6900e-01]
  [1.4200e-01]
  [1.3600e-01]
  [2.1800e-01]
  [2.0700e-01]
  [1.2300e-01]
  [1.1900e-01]
  [4.7400e-01]]]
Training labels samples:
[0 0]
Test data samples:
[[[4.8302e+04]
  [3.5800e-01]
  [4.1300e-01]
  [3.9200e-01]
  [2.4680e+00]
  [1.9000e-01]
  [2.0700e-01]
  [2.6800e-01]
  [2.5300e-01]
  [1.0700e-01]
  [1.4100e-01]
  [1.4100e-01]
  [3.0

## Data batching

In [12]:
def batch_client_data(client_data, batch_size=32):
    """Batch both training and test data for a client.

    Args:
        client_data: A dictionary containing 'train_data' and 'test_data' for the client.
        batch_size: The batch size for batching the datasets.

    Returns:
        A dictionary containing batched 'train_dataset' and 'test_dataset'.
    """
    # Batch both training and test data for a client
    train_data, train_labels = client_data['train_data']
    test_data, test_labels = client_data['test_data']

    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(len(train_labels)).batch(batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(batch_size)

    return {'train_dataset': train_dataset, 'test_dataset': test_dataset}

# Batch data for each client
clients_batched = {}
for client_name, data in clients.items():
    clients_batched[client_name] = batch_client_data(data)


# Local Training model (current: CTNN)

In [19]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Layer Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)

    # Adding
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    # x = layers.BatchNormalization()(x)  # Add BatchNormalization
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
    l2_reg=0.01  # L2 regularization factor, adjust as needed
    # attention_dropout=0 # Adding dropout to attention mechanism
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        # x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dense(dim, activation="relu", kernel_regularizer=l2(l2_reg))(x)  # L2 regularization added
        x = layers.BatchNormalization()(x)  # Add BatchNormalization
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    return keras.Model(inputs, outputs)


# Learning rate schedule
initial_learning_rate = 5e-3
lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)

In [20]:
class CTNN:
    def build(shape, classes):
        print('-----Transformer Neural Network:')

        # Model creation
        model = build_model(
            input_shape = X_train.shape[1:],
            head_size=128,  # change head size from 256 to 128 -> see how training time and acccuracy vary
            num_heads=4,
            ff_dim=4,
            num_transformer_blocks=1,
            mlp_units=[128],
            mlp_dropout=0.4,
            dropout=0.25,
            l2_reg=0.01  # L2 regularization factor, adjust as needed
            # attention_dropout=0.1
        )
        return model

# Global training model

In [15]:
def weighted_average(metrics, weights):
    """Function to compute the weighted average of performance metrics across all clients"""
    weighted_metrics = {}
    total_weight = sum(weights)
    for metric in metrics[0].keys():  # Assuming all metrics dicts have the same keys
        weighted_sum = sum(metric_value * weight for metric_value, weight in zip([m[metric] for m in metrics], weights))
        weighted_metrics[metric] = weighted_sum / total_weight
    return weighted_metrics

In [16]:
print(X_train.shape[1:])

(24, 1)


# Updated version of local_test and global platform for better output print (try it)

In [17]:
def local_test(model, test_dataset, client_name, comm_round, batch_size=32):
    # Initialize lists to store true labels and predictions
    true_labels = []
    predictions = []

    # Iterate through the test dataset batches
    for X_batch, y_batch in test_dataset:
        # Ensure the data is in the correct format and predict
        X_batch = tf.cast(X_batch, tf.float32)
        y_pred_batch = model.predict(X_batch, verbose=0)  # Suppress verbose output
        # Assuming binary classification
        y_pred_batch = (y_pred_batch > 0.5).astype(int).flatten()
        predictions.extend(y_pred_batch)
        true_labels.extend(y_batch.numpy().flatten())

    # Calculate and display metrics, ensure metrics are compatible with your labels and predictions
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average = 'weighted')  # Or 'binary' if it's a binary classification
    auc = roc_auc_score(true_labels, predictions)
    rmse = mean_squared_error(true_labels, predictions, squared = False)
    cm = confusion_matrix(true_labels, predictions)

    # Format the output
    print(f"Client {client_name} - Round {comm_round}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"AUC: {auc}")
    print(f"RMSE: {rmse}")
    print("Confusion Matrix:")
    for row in cm:
        print(row)
    print("\n")  # Add an extra newline for better separation

    return accuracy, f1, auc, rmse, cm

In [ ]:
# Initialize variables for federated training
comms_round = 10
client_names = list(clients_batched.keys())
client_accuracies = {client_name: [] for client_name in client_names}
client_rmse = {client_name: [] for client_name in client_names}
average_accuracies = []
average_rmse = []

# Initialize global weights
initial_global_model = CTNN()
global_weights = initial_global_model.build(X_train.shape[1:]).get_weights()

# Start the federated training loop
for comm_round in range(comms_round):
    scaled_local_weight_list = []
    round_accuracies = []
    round_rmse = []
    local_data_sizes = []

    for client_name in client_names:
        # Initialize and train local model
        smlp_local = CTNN()
        local_model = smlp_local.build(X_train.shape[1:])  # Adjust dimensions as necessary
        # local_model.compile(loss= 'binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 5e-3), metrics= ['accuracy'])
        local_model.compile(loss= 'binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics= ['accuracy'])
        # local_model.compile(loss= 'binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 1e-3), metrics= ['accuracy']) # decrease the learning rate to see whether it improves the accuracy dropping situation
        # local_model.compile(loss= 'binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 5e-4), metrics= ['accuracy'])

        # If we have previous global weights, apply them to the local model
        if global_weights:
            local_model.set_weights(global_weights)

        # Fit local model with client's training data
        local_model.fit(clients_batched[client_name]['train_dataset'], epochs=2, verbose=0) # increase the lcoal training epoch from 1 to 2 to see the changes

        # Perform local testing and get metrics
        accuracy, f1, auc, rmse, cm = local_test(local_model, clients_batched[client_name]['test_dataset'], client_name, comm_round)

        client_accuracies[client_name].append(accuracy)
        client_rmse[client_name].append(rmse)
        round_accuracies.append(accuracy)
        round_rmse.append(rmse)

        # Scale the local model weights for aggregation
        local_data_size = list(clients_batched[client_name]['train_dataset'].unbatch().as_numpy_iterator()).__len__()
        local_data_sizes.append(local_data_size)
        scaling_factor = local_data_size / sum(local_data_sizes)  # Adjust based on actual data size
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # Clear session to free memory
        K.clear_session()

    # Update global weights
    global_weights = sum_scaled_weights(scaled_local_weight_list)

    # Calculate average metrics for this round and store them
    average_accuracies.append(sum(round_accuracies) / len(client_names))
    average_rmse.append(sum(round_rmse) / len(client_names))

# After all rounds, plot the metrics
plt.figure(figsize=(14, 5))

# Plotting average accuracies
plt.subplot(1, 2, 1)
plt.plot(range(comms_round), average_accuracies, marker='x', label='Average Accuracy')
for client_name in client_names:
    plt.plot(range(comms_round), client_accuracies[client_name], marker='o', linestyle='dashed', label=f'Client {client_name}')
plt.title('Average Accuracy vs Client Accuracies over Communication Rounds')
plt.xlabel('Communication Rounds')
plt.ylabel('Accuracy')
plt.legend()

# Plotting average RMSE
plt.subplot(1, 2, 2)
plt.plot(range(comms_round), average_rmse, marker='x', label='Average RMSE')
for client_name in client_names:
    plt.plot(range(comms_round), client_rmse[client_name], marker='o', linestyle='dashed', label=f'Client {client_name}')
plt.title('Average RMSE vs Client RMSE over Communication Rounds')
plt.xlabel('Communication Rounds')
plt.ylabel('RMSE')
plt.legend()

plt.tight_layout()
plt.show()


-----Transformer Neural Network:
-----Transformer Neural Network:
Client client_1 - Round 0:
Accuracy: 0.9502844883872773
F1 Score: 0.9460525750270697
AUC: 0.8342661691542288
RMSE: 0.22296975492815768
Confusion Matrix:
[18226     0]
[1066 2150]


-----Transformer Neural Network:
Client client_2 - Round 0:
Accuracy: 0.9366197183098591
F1 Score: 0.9292692883321588
AUC: 0.7887126865671642
RMSE: 0.25175440748900674
Confusion Matrix:
[18226     0]
[1359 1857]


-----Transformer Neural Network:
Client client_3 - Round 0:
Accuracy: 0.9491185523738457
F1 Score: 0.9446611123523272
AUC: 0.8303793532338308
RMSE: 0.22556916373067104
Confusion Matrix:
[18226     0]
[1091 2125]


-----Transformer Neural Network:
Client client_1 - Round 1:
Accuracy: 0.940490625874452
F1 Score: 0.9341333099297271
AUC: 0.8016169154228856
RMSE: 0.24394543268023688
Confusion Matrix:
[18226     0]
[1276 1940]


-----Transformer Neural Network:
Client client_2 - Round 1:
Accuracy: 0.9374125547989927
F1 Score: 0.93027301897